In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_samples
)
plt.style.use("seaborn-v0_8-whitegrid")

sns.set_context("talk")
sns.set_palette("deep")

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.titlesize"] = 18
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12

In [ ]:
# Load PCA transformed data (4 components)
X_pca = pd.read_csv("pca_transformed_data.csv")

print("Shape:", X_pca.shape)
X_pca.head()

In [ ]:
kmeans = KMeans(
    n_clusters=4,
    random_state=42,
    n_init=20
)

cluster_labels = kmeans.fit_predict(X_pca)

X_pca['Cluster'] = cluster_labels

print("Cluster counts:")
print(X_pca['Cluster'].value_counts())

In [ ]:
original_df = pd.read_csv("final_feature_engineered.csv")

# attach cluster labels
original_df['Cluster'] = cluster_labels

print(original_df.shape)

cluster_names = {
    0: "Low Cost Sustainable Segment",
    1: "Climate Sensitive Segment",
    2: "Operational Optimization Opportunity",
    3: "High HVAC High Renewable Segment"
}

original_df['Cluster_Name'] = original_df['Cluster'].map(cluster_names)

In [ ]:
X_features = X_pca.drop(columns=['Cluster'])

sil_score = silhouette_score(X_features, cluster_labels)
ch_score = calinski_harabasz_score(X_features, cluster_labels)
db_score = davies_bouldin_score(X_features, cluster_labels)

print("Silhouette:", round(sil_score,4))
print("Calinski-Harabasz:", round(ch_score,2))
print("Davies-Bouldin:", round(db_score,4))

In [ ]:
# ==============================
# Stability Test
# ==============================

stability_scores = []

for seed in range(10):
    km = KMeans(n_clusters=4, random_state=seed, n_init=20)
    labels_temp = km.fit_predict(X_features)
    score = silhouette_score(X_features, labels_temp)
    stability_scores.append(score)

print("Mean Silhouette:", np.mean(stability_scores))
print("Std Deviation:", np.std(stability_scores))

In [ ]:
sample_sil_values = silhouette_samples(X_features, cluster_labels)

fig, ax = plt.subplots(figsize=(10,7))

y_lower = 10

for i in range(4):
    cluster_sil_values = sample_sil_values[cluster_labels == i]
    cluster_sil_values.sort()
    size_cluster_i = cluster_sil_values.shape[0]
    y_upper = y_lower + size_cluster_i

    ax.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        cluster_sil_values,
        alpha=0.7
    )

    ax.text(-0.05, y_lower + 0.5*size_cluster_i, str(i))
    y_lower = y_upper + 10

ax.axvline(x=sil_score, color="black", linestyle="--", linewidth=2)

ax.set_title("Silhouette Analysis for K = 4")
ax.set_xlabel("Silhouette Coefficient")
ax.set_ylabel("Cluster")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,7))

sns.scatterplot(
    data=X_pca,
    x="PC1",
    y="PC2",
    hue="Cluster",
    palette="Set2",
    alpha=0.6,
    edgecolor=None
)

plt.title("Cluster Separation in PCA Space")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Cluster", bbox_to_anchor=(1.05,1))
plt.tight_layout()
plt.show()

In [ ]:
cluster_counts = original_df['Cluster_Name'].value_counts()

plt.figure(figsize=(10,6))
sns.barplot(
    x=cluster_counts.values,
    y=cluster_counts.index,
    palette="viridis"
)

plt.title("Cluster Distribution")
plt.xlabel("Number of Buildings")
plt.ylabel("")
plt.tight_layout()
plt.show()

In [ ]:
centroid_features = [
    'Energy_Intensity',
    'HVAC_Ratio',
    'Lighting_Ratio',
    'Occupancy_Adjusted_Energy',
    'Renewable_Ratio',
    'Carbon_Load',
    'Efficiency_Gap',
    'Savings_Gap',
    'Grid_Stress',
    'Temp_Deviation'
]

cluster_centroids = original_df.groupby('Cluster')[centroid_features].mean()
cluster_centroids

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cluster_means_scaled = pd.DataFrame(
    scaler.fit_transform(cluster_centroids),
    columns=cluster_centroids.columns,
    index=cluster_centroids.index
)

In [ ]:
def plot_radar(cluster_id):
    values = cluster_means_scaled.loc[cluster_id].values
    labels = cluster_means_scaled.columns

    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
    values = np.concatenate((values, [values[0]]))
    angles = np.concatenate((angles, [angles[0]]))

    fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(polar=True))
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)
    ax.set_ylim(-2, 2)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, size=8)
    ax.set_title(f"Cluster {cluster_id} Profile",pad =20)
    plt.show()

for c in cluster_means_scaled.index:
    plot_radar(c)

ANOVA

In [ ]:
from scipy.stats import f_oneway

anova_results = {}

for feature in centroid_features:
    groups = [
        original_df[original_df['Cluster'] == c][feature]
        for c in original_df['Cluster'].unique()
    ]
    
    f_stat, p_value = f_oneway(*groups)
    
    anova_results[feature] = {
        'F-statistic': f_stat,
        'p-value': p_value
    }

anova_df = pd.DataFrame(anova_results).T
anova_df

Energy impact cost analysis

In [ ]:
original_df['Energy_Cost'] = (
    original_df['Energy Consumption (kWh)'] *
    original_df['Energy Price ($/kWh)']
)

cost_analysis = original_df.groupby('Cluster_Name')['Energy_Cost'].mean()

plt.figure(figsize=(10,6))

sns.barplot(
    data=original_df,
    x="Cluster_Name",
    y="Energy_Cost",
    estimator=np.mean,
    errorbar=None,
    palette="coolwarm"
)

plt.xticks(rotation=30)
plt.title("Average Energy Cost by Cluster")
plt.ylabel("Average Cost ($)")
plt.xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
carbon_analysis = original_df.groupby('Cluster')['Carbon_Load'].mean()
carbon_analysis

In [ ]:
efficiency_analysis = original_df.groupby('Cluster')[['Efficiency_Gap','Savings_Gap']].mean()

efficiency_analysis

In [ ]:
# ==============================
# Potential Savings Analysis
# ==============================

# Estimated potential savings per building
original_df['Potential_Savings_Value'] = (
    original_df['Savings_Gap'] *
    original_df['Energy Consumption (kWh)']
)

cluster_savings = original_df.groupby('Cluster_Name')[
    'Potential_Savings_Value'
].mean()

plt.figure(figsize=(10,6))

sns.barplot(
    data=original_df,
    x="Cluster_Name",
    y="Potential_Savings_Value",
    estimator=np.mean,
    errorbar=None,
    palette="magma"
)

plt.xticks(rotation=30)
plt.title("Average Potential Savings by Cluster")
plt.ylabel("Estimated Savings Value")
plt.xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
cluster_percent = (
    original_df['Cluster_Name']
    .value_counts(normalize=True) * 100
)

print(cluster_percent)

In [ ]:
if 'hour_sin' in original_df.columns and 'hour_cos' in original_df.columns:

    hourly_profile = original_df.groupby('Cluster')[['hour_sin', 'hour_cos']].mean()

    hourly_profile['approx_hour'] = (
        np.arctan2(hourly_profile['hour_sin'], hourly_profile['hour_cos']) 
        / (2*np.pi) * 24
    ) % 24

    print(hourly_profile[['approx_hour']])

In [ ]:
# ==============================
# Recommendation Table
# ==============================

recommendations = pd.DataFrame({
    "Cluster": [0,1,2,3],
    "Cluster_Name": [
        "Low Cost Sustainable Segment",
        "Climate Sensitive Segment",
        "Operational Optimization Opportunity",
        "High HVAC High Renewable Segment"
    ],
    "Recommended Actions": [
        "Maintain performance, predictive monitoring, minor HVAC tuning",
        "Implement climate-adaptive HVAC control & insulation upgrades",
        "Deploy occupancy-driven automation & HVAC scheduling optimization",
        "Improve HVAC efficiency & integrate demand-response systems"
    ],
    "Priority_Level": [
        "Medium",
        "High",
        "Very High",
        "Medium"
    ]
})

recommendations

In [ ]:
# Ensure cost exists
original_df['Energy_Cost'] = (
    original_df['Energy Consumption (kWh)'] *
    original_df['Energy Price ($/kWh)']
)

cost_analysis_cluster = original_df.groupby('Cluster')['Energy_Cost'].mean()
carbon_analysis_cluster = original_df.groupby('Cluster')['Carbon_Load'].mean()
savings_analysis_cluster = original_df.groupby('Cluster')['Potential_Savings_Value'].mean()
cluster_percent = original_df['Cluster'].value_counts(normalize=True) * 100

In [ ]:
final_summary = cluster_centroids.copy()

final_summary['Cluster_%'] = cluster_percent
final_summary['Avg_Energy_Cost'] = cost_analysis_cluster
final_summary['Avg_Carbon_Load'] = carbon_analysis_cluster
final_summary['Avg_Potential_Savings'] = savings_analysis_cluster

final_summary = final_summary.sort_index()

final_summary

In [ ]:
plt.figure(figsize=(12,8))

sns.heatmap(
    final_summary.drop(columns=["Cluster_%"]),
    annot=True,
    fmt=".2f",
    cmap="YlGnBu"
)

plt.title("Cluster Comparative Summary Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
plt.savefig("cluster_distribution.png", dpi=300, bbox_inches='tight')

In [ ]:
final_summary.to_csv("final_cluster_summary.csv")
recommendations.to_csv("cluster_recommendations.csv")